In [ ]:
# default_exp utils

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Utils module

> Provides utility functions and classes

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script
notebook2script()

Converted 00_init.ipynb.
Converted 01_graph.ipynb.
Converted 02_tree.ipynb.
Converted 03_synteny.ipynb.
Converted 04_utils.ipynb.
Converted 05_export.ipynb.
Converted index.ipynb.


In [ ]:
#export
from collections.abc import Iterable
from copy import deepcopy
import json
import numpy as np
import os

In [ ]:
#export
def pathConvert(inputPath,suffix=''):
    outputPath = os.path.dirname(inputPath)
    outputName = '.'.join(os.path.splitext(os.path.basename(inputPath))[:-1])+suffix
    return outputPath,outputName

In [ ]:
#export
def checkZoomLevels(zoomLevels):
    '''
    Check that each previous zoom level is factor of next one
    '''
    _zoomLevels = np.array(zoomLevels)
    div = _zoomLevels[1:]/_zoomLevels[:-1]
    return not np.any(div - div.astype(np.int))

In [ ]:
#export
def adjustZoomLevels(zoomLevels):
    '''
    If there is no zoom level 1, adds it to the list.
    '''
    if not checkZoomLevels(zoomLevels):
        raise ValueError('Zoom level list is incorrect. Each next level should have previous one as factor.')
    if min(zoomLevels)>1:
        zoomLevels = [1] + zoomLevels
    return zoomLevels

[1, 4, 8, 16]

In [ ]:
#export
# https://stackoverflow.com/questions/50916422/python-typeerror-object-of-type-int64-is-not-json-serializable
# Class for encoding np types to JSON
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, set):
            return list(obj)
        else:
            return super(NpEncoder, self).default(obj)


In [ ]:
#export
class bidict(dict):
    '''
    Here is a class for a bidirectional dict, inspired by Finding key from value in Python 
    dictionary and modified to allow the following 2) and 3).

    Note that :

    1) The inverse directory bd.inverse auto-updates itself when the standard dict bd is modified.
    2) The inverse directory bd.inverse[value] is always a list of key such that bd[key] == value.
    3) Unlike the bidict module from https://pypi.python.org/pypi/bidict, here we can have 2 keys 
    having same value, this is very important.
    4) After modification, values in the "forward" (not inversed) dict can be lists (or any 
    iterables theoretically, but only list was tested). 
    
    For implementing 4), new method `add` was introduced. If d[key].append(value) attempted, the link 
    between main and inversed dict will be broken. Method `add` can accept both 
    
    Credit: 
    Implemented as an answer to 
    https://stackoverflow.com/questions/3318625/how-to-implement-an-efficient-bidirectional-hash-table
    by Basj (https://stackoverflow.com/users/1422096/basj).
    '''
    def __init__(self, *args, **kwargs):
        super(bidict, self).__init__(*args, **kwargs)
        self.inverse = {}
        for key, value in self.items():
            if isinstance(value,Iterable):
                for v in value:
                    self.inverse.setdefault(v,[]).append(key) 
            else:
                self.inverse.setdefault(value,[]).append(key) 

    def __setitem__(self, key, value):
        
        if key in self:
            keyV = self[key]
            if isinstance(keyV,Iterable):
                for v in keyV:
                    self.inverse[v].remove(key)
            else:
                self.inverse[keyV].remove(key)
        super(bidict, self).__setitem__(key, value)
        if isinstance(value,Iterable):
            for v in value:
                self.inverse.setdefault(v,[]).append(key)        
        else:
            self.inverse.setdefault(value,[]).append(key)        

    def __delitem__(self, key):
        value = self[key]
        if isinstance(value,Iterable):
            for v in value:
                self.inverse.setdefault(v,[]).remove(key)
                if v in self.inverse and not self.inverse[v]: 
                    del self.inverse[v]
        else:
            self.inverse.setdefault(value,[]).remove(key)
            if value in self.inverse and not self.inverse[value]: 
                del self.inverse[value]
        super(bidict, self).__delitem__(key)
    
    def add(self,key,value):
        valKey = set(self.setdefault(key,[]))
        
        if isinstance(value,Iterable):
            valKey = valKey.union(value)
        else:
            valKey.add(value)
        
        self[key] = list(valKey)

In [ ]:
#export
def pathFileToPathDict(filePath,directional=True):
    paths = {}
    with open(filePath) as f:
        for line in f:
            pathName,pathNodeList = line.strip(' \n\t').split(':')
            if directional:
                paths[pathName] = pathNodeList.strip().split(',')
            else:
                paths[pathName] = [f'{node}+' for node in pathNodeList.strip().split(',')]
    
    return paths